In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('../input/digit-recognizer/train.csv')

In [3]:
data = np.array(data)
m, n = data.shape #n = features + 1
np.random.shuffle(data)


data_dev = data[0:1000].T

Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255
# dev is for first 1000 examples

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255


M is the examples
N is the features +1
the one is the actual value

In [4]:
def init_params():
    W1 = np.random.rand(10,784) - 0.5
    b1 = np.random.rand(10,1) - 0.5
    
    W2 = np.random.rand(10,10) - 0.5
    b2 = np.random.rand(10,1) - 0.5
    return W1,b1,W2,b2
# WHAT is so special about the - 0.5
def ReLU(Z):
#     if(x>0):
#         return x
#     else:
#         return 0
    return np.maximum(0, Z)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1,b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1,A1,Z2,A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size,Y.max()+1))
    one_hot_Y[np.arange(Y.size),Y]=1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0
    

def back_prop(Z1,A1,Z2,A2,W2,X,Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = (1/m)*(dZ2.dot(A1.T))
    db2 = (1/m)*(np.sum(dZ2))
    
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = (1/m) * (dZ1.dot(X.T))
    db1 = (1/m)* (np.sum(dZ1))
    return dW1,db1,dW2,db2

def update_params(W1,b1,W2,b2, dW1,db1,dW2,db2, alpha):
    W1 = W1 - alpha* dW1
    b1 = b1 - alpha*db1
    
    W2 = W2 - alpha* dW2
    b2 = b2 - alpha*db2
    
    return W1,b1, W2,b2
    
    
    

In [5]:
def get_predictions(A2):
    return np.argmax(A2,0)

def get_accuracy(predictions,  Y):
    print(predictions, Y)
    return np.sum(predictions==Y) / Y.size


def gradient_descent(X,Y,iterations,alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1,A1,Z2,A2 = forward_prop(W1, b1, W2, b2,X)
        dW1,db1,dW2,db2 = back_prop(Z1,A1,Z2,A2,W2,X,Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i%100 == 0:
            print("iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
            print(W1)
    return W1, b1, W2, b2
    

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 2000, 0.1)


iteration:  0
[4 2 4 ... 8 8 2] [5 9 1 ... 5 1 5]
Accuracy:  0.13046341463414635
[[ 0.4841271  -0.49602308 -0.18718026 ...  0.47634484 -0.30261529
   0.40066011]
 [-0.38618193  0.45348465 -0.10269714 ...  0.30123709 -0.05710606
   0.46131541]
 [-0.33045458  0.23160533 -0.24182798 ... -0.08488378 -0.4975652
   0.45505199]
 ...
 [-0.11189321 -0.03389787 -0.11928824 ...  0.4652645   0.10758489
   0.07892747]
 [-0.06318919  0.27349401 -0.15128678 ...  0.46156169  0.49068417
   0.3985536 ]
 [ 0.01427409 -0.24922785  0.19901026 ... -0.16205425 -0.39061813
  -0.07030732]]
iteration:  100
[8 6 1 ... 8 8 5] [5 9 1 ... 5 1 5]
Accuracy:  0.641219512195122
[[ 0.4841271  -0.49602308 -0.18718026 ...  0.47634484 -0.30261529
   0.40066011]
 [-0.38618193  0.45348465 -0.10269714 ...  0.30123709 -0.05710606
   0.46131541]
 [-0.33045458  0.23160533 -0.24182798 ... -0.08488378 -0.4975652
   0.45505199]
 ...
 [-0.11189321 -0.03389787 -0.11928824 ...  0.4652645   0.10758489
   0.07892747]
 [-0.06318919  0.27